In [1]:
from sunpy.net import jsoc, attrs as a
import astropy.units as u
import numpy as np
from glob import glob
import time
from sunpy.net.jsoc.jsoc import NotExportedError

cadence = 1*u.min
client = jsoc.JSOCClient()

response = client.search(a.jsoc.Time('2013/5/13 15:55', '2013/5/13 18:55'),  #a.jsoc.Time('2010/1/1 12:00', '2013/7/18 12:00'),
                         a.jsoc.Series('hmi.Ic_45s'),
                         a.jsoc.Notify("brettmorris21@gmail.com"),
                         a.vso.Sample(cadence))
print(response)
requests = client.request_data(response)
print(requests)
time.sleep(30)
res = client.get_request(requests, path='hmi_twospots/.', max_conn=10, overwrite=True)
res.wait(progress=True)

/Users/bmmorris/anaconda/lib/python3.5/site-packages/glymur/config.py:171: UserWarning: Neither the openjp2 nor the openjpeg library could be loaded.  
  warnings.warn(msg)


RuntimeError: module compiled against API version 0xc but this version of numpy is 0xa

In [2]:
from astropy.io import fits

paths = glob('hmi_twospots/*.fits')

f = fits.open(paths[0], memmap=False)
f[1].verify('silentfix')
shape = f[1].data.shape

In [3]:

import h5py
f = h5py.File('hmi_twospots/archive.hdf5', 'w')

dset = f.create_dataset("images", shape=(len(paths), shape[0], shape[1]), dtype=int, 
                        compression='lzf')

In [4]:
hdu = fits.open(paths[0], memmap=False)
hdu[1].verify('silentfix')
image = hdu[1].data

dset[0, :, :] = image

In [5]:
from astropy.utils.console import ProgressBar

with ProgressBar(len(paths), ipython_widget=True) as bar: 
    for i, path in enumerate(paths): 
        hdu = fits.open(path, memmap=False)
        hdu[1].verify('silentfix')
        image = hdu[1].data

        dset[i, :, :] = image
        bar.update()
        f.flush()
    f.close()

FloatProgress(value=0.0)